### 1. import packages

In [19]:
from selenium import webdriver
from selenium.common import exceptions
import sys
import time
import pandas as pd

# selenium has to be installed using Anaconda prompt here.
# selenium needs downloading "chromedriver"

### 2. Def a function for scraping

In [20]:
def scrape_comment(url):
    
    # Note: replace argument with absolute path to the driver executable.
    driver = webdriver.Chrome('C:/Users/80250678/Desktop/chromedriver')

    driver.get(url)
    driver.maximize_window()
    time.sleep(5)

    try:
        comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
    except exceptions.NoSuchElementException:
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)

    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)

    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        time.sleep(2)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    try:
        # Extract the elements storing the usernames and comments.
        username_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
        comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
    except exceptions.NoSuchElementException:
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)        
    
    for comment in comment_elems:
        comment_list.append(comment.text)

    driver.close()

### 3. import youtube links

In [21]:
mv = pd.read_csv("C:/Users/80250678/Desktop/kpop_music_videos.csv")

In [50]:
artist_mv = mv[["Date","Artist","Song Name","Video"]]
artist_mv

,Date,Artist,Song Name,Video
0,2020-05-22,Agust D,Daechwita,https://youtu.be/qGjAWJ2zWWI
1,2020-05-21,Yubin,yaya (Me Time),https://youtu.be/BJD0arHF_5c
2,2020-05-21,OnlyOneOf,Angel (prod. GRAY),https://youtu.be/vVTo8p72FxQ
3,2020-05-20,Ryu Sujeong,Tiger Eyes,https://youtu.be/aE6curPGQRY
4,2020-05-20,Crush,Mayday feat. Joy (Red Velvet),https://youtu.be/29ycT6fA-Rs
...,...,...,...,...
3767,1994-08-08,Seo Taiji and Boys,Eternity,https://youtu.be/74usU0gB7r0
3768,1993-06-06,Seo Taiji and Boys,Anyhow Song,https://youtu.be/L-AxO7EPU8c
3769,1993-06-06,Seo Taiji and Boys,To You,https://youtu.be/cxIAiAUyvXo
3770,1992-03-23,Seo Taiji and Boys,I Know,https://youtu.be/OEDHEzs5kyk


### 4. iteration for each video

In [31]:
# get video list
video_list = artist_mv["Video"].to_list()

In [55]:
#create a vacant dataframe
mv_comment = pd.DataFrame(columns=["Video","Comment"])

for link in video_list:
    comment_list=[]
    scrape_comment(link)
        # create a table for videos and comments
    mid = pd.DataFrame(columns=["Video","Comment"])
    mid["Comment"]=pd.Series(comment_list)
    mid["Video"]=link
    mv_comment = pd.concat([mv_comment,mid])

In [56]:
artist_mv_comment = pd.merge(artist_mv,mv_comment,on="Video",how="inner")

,Date,Artist,Song Name,Video,Comment_x,Comment_y
0,2020-05-22,Agust D,Daechwita,https://youtu.be/qGjAWJ2zWWI,Agust D is hotter than the sun.,NaN
1,2020-05-22,Agust D,Daechwita,https://youtu.be/qGjAWJ2zWWI,"El video, la letra, Yoongi, TODO ES ARTE",NaN
2,2020-05-22,Agust D,Daechwita,https://youtu.be/qGjAWJ2zWWI,Suga: I am a king \nMe: Can't I work in the pa...,NaN
3,2020-05-22,Agust D,Daechwita,https://youtu.be/qGjAWJ2zWWI,I did realize how much I like yoongi with long...,NaN
4,2020-05-22,Agust D,Daechwita,https://youtu.be/qGjAWJ2zWWI,0:59 \nJin y Jungkok queremos aparecer en tu v...,NaN
...,...,...,...,...,...,...
6125,1994-08-08,Seo Taiji and Boys,Eternity,https://youtu.be/74usU0gB7r0,NaN,NaN
6126,1993-06-06,Seo Taiji and Boys,Anyhow Song,https://youtu.be/L-AxO7EPU8c,NaN,NaN
6127,1993-06-06,Seo Taiji and Boys,To You,https://youtu.be/cxIAiAUyvXo,NaN,NaN
6128,1992-03-23,Seo Taiji and Boys,I Know,https://youtu.be/OEDHEzs5kyk,NaN,NaN
